<h1> Parameter Tunning with MinMax
    
    RandomizedCV for: hidden layers, 
    neurons, optimizer, kernel initializer and bias initializer

# Importing libraries:

In [1]:
!pip install pandas
!pip install sklearn

import pandas as pd;
import numpy as np;
import sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict;
from sklearn.preprocessing import MinMaxScaler;
from sklearn import metrics;
from sklearn.model_selection import TimeSeriesSplit;

# Import data; set X and y; fill nan values and split in test and training  data:

In [2]:
# import data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)
bin_dataset = pd.read_csv('Spike_binary_1std.csv', index_col = 0)
bin_dataset = bin_dataset['spike_occurance']

# filter max values for offer if required
print(data.Offers.max()) #max is 2500... no need to filter max values

# 2017 & 2018 data
data = data.loc[data.index > 2018060000, :]
bin_dataset = bin_dataset.loc[bin_dataset.index > 2018060000]

# reset index
data.reset_index(drop = True, inplace = True)
bin_dataset.reset_index(drop = True, inplace = True)

# Divide features and labels
X = data.iloc[:, 0:15]
y = bin_dataset

# fill nan values
X.fillna(method = 'ffill', inplace = True)

X = X.astype('float64')
X = X.round(20)

# divide data into train and test with 20% test data
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.1, shuffle=False)

2500.0


/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


# Apply feature scaling:

In [3]:
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Importing the Keras libraries and packages:

In [4]:
import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
import keras.optimizers
from keras import initializers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

Using TensorFlow backend.


# Create regressor:

In [5]:
# possible debug
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

def regressor_tunning(n_hidden = 1, 
                      n_neurons = 11, 
                      optimizer = 'RMSprop', 
                      kernel_initializer="he_normal",
                      bias_initializer= initializers.Ones():
    model = Sequential()
    model.add(Dense(output_dim = n_neurons, 
                    input_dim = 15))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(Dense(output_dim = 1, 
                    activation = 'sigmoid'))
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
    return model

# Apply Randomized tunning for the whole ANN:

In [6]:
regressor = KerasRegressor(build_fn = regressor_tunning)

# Dictionary to include the parameters
parameters = {'n_hidden': [1, 2, 3, 4, 6],
              'n_neurons': np.arange(10,100, 5),
              'bias_initializer':[initializers.Zeros(),
                                 initializers.Ones()],
              'kernel_initializer': ['glorot_uniform',
                                     'he_normal',
                                     'he_uniform'],
              'optimizer': [keras.optimizers.RMSprop(), 
                            keras.optimizers.Adam(), 
                            keras.optimizers.Nadam(),
                            keras.optimizers.Adamax()]
               }

tscv = TimeSeriesSplit(n_splits = 7)

# add some early stopping
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='mse', patience = 15)

rnd_search_cv = RandomizedSearchCV(estimator = regressor,
                                   param_distributions = parameters,
                                   scoring = 'neg_mean_squared_error',
                                   n_iter = 20,
                                   n_jobs = -1,
                                   cv = tscv)

# checkpoints:
# early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
# checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5")

rnd_search_cv.fit(X_train, y_train, batch_size = 32, epochs = 60, callbacks=[early_stopping])


/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=15, units=45)`
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
  from ipykernel import kernelapp as app


Epoch 1/60
1850/1850 [==============================] - 6s 3ms/step - loss: 11769.0269 - mse: 11769.0273 - mae: 103.6678
Epoch 2/60
1850/1850 [==============================] - 1s 731us/step - loss: 8459.8813 - mse: 8459.8818 - mae: 85.9329
Epoch 3/60
1850/1850 [==============================] - 1s 729us/step - loss: 3721.4481 - mse: 3721.4487 - mae: 50.9239
Epoch 4/60
1850/1850 [==============================] - 1s 711us/step - loss: 1203.7260 - mse: 1203.7260 - mae: 23.5375
Epoch 5/60
1850/1850 [==============================] - 1s 665us/step - loss: 1039.3145 - mse: 1039.3145 - mae: 22.4298
Epoch 6/60
1850/1850 [==============================] - 1s 621us/step - loss: 1035.4982 - mse: 1035.4982 - mae: 22.4625
Epoch 7/60
1850/1850 [==============================] - 1s 647us/step - loss: 1025.7811 - mse: 1025.7812 - mae: 22.1371
Epoch 8/60
1850/1850 [==============================] - 1s 663us/step - loss: 1025.0591 - mse: 1025.0590 - mae: 22.1933
Epoch 9/60
1850/1850 [=================

/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=15, units=45)`
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
  from ipykernel import kernelapp as app


Epoch 1/60
3699/3699 [==============================] - 7s 2ms/step - loss: 9649.9854 - mse: 9649.9893 - mae: 91.6820
Epoch 2/60
3699/3699 [==============================] - 2s 654us/step - loss: 2219.1847 - mse: 2219.1848 - mae: 33.6190
Epoch 3/60
3699/3699 [==============================] - 2s 633us/step - loss: 1142.7195 - mse: 1142.7196 - mae: 23.1229
Epoch 4/60
3699/3699 [==============================] - 2s 643us/step - loss: 1135.2183 - mse: 1135.2184 - mae: 22.9910
Epoch 5/60
3699/3699 [==============================] - 2s 632us/step - loss: 1132.3443 - mse: 1132.3442 - mae: 23.0339
Epoch 6/60
3699/3699 [==============================] - 3s 717us/step - loss: 1127.4828 - mse: 1127.4828 - mae: 22.9512
Epoch 7/60
3699/3699 [==============================] - 3s 719us/step - loss: 1126.9325 - mse: 1126.9327 - mae: 22.8807
Epoch 8/60
3699/3699 [==============================] - 3s 698us/step - loss: 1123.9382 - mse: 1123.9384 - mae: 22.8232
Epoch 9/60
3699/3699 [====================

/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=15, units=45)`
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
  from ipykernel import kernelapp as app


...

/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=15, units=90)`
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
  from ipykernel import kernelapp as app


Epoch 1/60
5548/5548 [==============================] - 10s 2ms/step - loss: 3557.5606 - mse: 3557.5583 - mae: 39.5600
Epoch 2/60
5548/5548 [==============================] - 4s 756us/step - loss: 1687.2656 - mse: 1687.2656 - mae: 25.2353
Epoch 3/60
5548/5548 [==============================] - 4s 747us/step - loss: 1671.1297 - mse: 1671.1298 - mae: 25.1364
Epoch 4/60
5548/5548 [==============================] - 5s 835us/step - loss: 1677.3429 - mse: 1677.3431 - mae: 25.1390
Epoch 5/60
5548/5548 [==============================] - 4s 805us/step - loss: 1655.5377 - mse: 1655.5375 - mae: 24.9724
Epoch 6/60
5548/5548 [==============================] - 5s 835us/step - loss: 1657.2286 - mse: 1657.2280 - mae: 24.9556
Epoch 7/60
5548/5548 [==============================] - 5s 869us/step - loss: 1654.7170 - mse: 1654.7174 - mae: 24.8216
Epoch 8/60
5548/5548 [==============================] - 5s 857us/step - loss: 1651.1994 - mse: 1651.1996 - mae: 24.8008
Epoch 9/60
5548/5548 [===================

/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=15, units=90)`
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
  from ipykernel import kernelapp as app


Epoch 1/60
7397/7397 [==============================] - 12s 2ms/step - loss: 3214.3567 - mse: 3214.3557 - mae: 36.9611
Epoch 2/60
7397/7397 [==============================] - 6s 775us/step - loss: 1608.5722 - mse: 1608.5718 - mae: 25.3637
Epoch 3/60
7397/7397 [==============================] - 7s 914us/step - loss: 1596.3080 - mse: 1596.3079 - mae: 25.2547
Epoch 4/60
7397/7397 [==============================] - 6s 777us/step - loss: 1590.2797 - mse: 1590.2800 - mae: 25.2646
Epoch 5/60
7397/7397 [==============================] - 6s 837us/step - loss: 1586.0242 - mse: 1586.0236 - mae: 25.1113
Epoch 6/60
7397/7397 [==============================] - 6s 838us/step - loss: 1585.0625 - mse: 1585.0624 - mae: 25.0852
Epoch 7/60
7397/7397 [==============================] - 6s 749us/step - loss: 1574.7479 - mse: 1574.7482 - mae: 25.0816
Epoch 8/60
7397/7397 [==============================] - 6s 814us/step - loss: 1571.8135 - mse: 1571.8135 - mae: 24.9807
Epoch 9/60
7397/7397 [===================

RuntimeError: Cannot clone object <keras.wrappers.scikit_learn.KerasRegressor object at 0x7f43a375a610>, as the constructor either does not set or modifies parameter optimizer

In [8]:
best_params_1 = rnd_search_cv.best_params_
best_score_1 = rnd_search_cv.best_score_
#results = rnd_search_cv.cv_results_

print("the best parameters are:{}".format(best_params_1))
print("the best score is:{}".format(best_score_1))
#print("results:".format(results))

the best parameters are:{'optimizer': <keras.optimizers.Adamax object at 0x7f43e1fdbfd0>, 'n_neurons': 50, 'n_hidden': 4, 'kernel_initializer': 'glorot_uniform', 'bias_initializer': <keras.initializers.Zeros object at 0x7f43b955cbd0>}
the best score is:-1995.0684018410414
